# 확률 모델과 최대 우도 추정
probability and Maximum Likelihood Estimation

- 확률 모델 : 변수 x가 파라미터 세타를 지니는 확률 분포(P(x|세타))로부터 생성된다고 가정하는 모델
    - x가 연속 변수일 경우 정규 분포가 될 수 있음
    - 이산 변수일 경우 베르누이 분포가 될 수 있음

- 우도: 서로 독립된 n개의 데이터가 주어졌을 때 다음과 같이 각 데이터의 확률 함수 값의 곱을 세타의 함수라고 하면 세타가 가능 정도가 되며 우도라고 부름
    - 또는 주어진 데이터가 모델에 얼마나 적합한지에 대한 조건부 확률(발생 확률을 나타내는 함수)

# 선형회귀 모델 구현

## 테스트 데이터 생성 및 파라미터 학습을 위한 변수 정의

In [1]:
import torch

# 참의 계수
w_true=torch.Tensor([1,2,3])

#X 데이터, 절편을 회귀계수에 포함, X의 최초 차원에|1 추가
X=torch.cat([torch.ones(100,1), torch.randn(100,2)],1)

#참의 계수와 각 X의 내적을 햏렬과 벡터의 곱으로 모아서 계산
y=torch.mv(X,w_true) + torch.randn(100)*0.5

#기울기 하강으로 최적화 하기 위해 파라미터 tensor를 난수로 초기화
w=torch.randn(3, requires_grad=True)
#학습률
learning_rate=0.1

## 경사 하강법으로 파라미터 최적화

In [2]:
losses=[]

#100회 반복
for e in range(100):
    w.grad=None # 전회 역전파로 계산된 경사값 초기화, optimizer zero_grad
    
    #y 값 예측
    y_pred=torch.mv(X,w)
    
    # MSE loss, w grad 계산
    loss=torch.mean((y-y_pred)**2)
    loss.backward()
    
    # 경사 갱신(계산 그래프 유지를 위해 .data만 갱신
    w.data=w.data-learning_rate*w.grad.data
    
    #수렴확인을 위한 loss기록
    losses.append(loss.item())

In [3]:
w

tensor([0.9178, 1.9964, 3.0660], requires_grad=True)

# nn, optim 모듈 사용

In [4]:
from torch import nn, optim

#선형층 작성, 절편이 회귀 계수에 포함되므로 입력차원을 3, 바이어스를 false
net=nn.Linear(in_features=3, out_features=1, bias=False)

# sgd 최적화기상에서 정의한 네트어크의 파라미터를 전달해 초기화
opti=optim.SGD(net.parameters(),lr=0.1)

#MSE loss
loss_fn=nn.MSELoss()

In [5]:
losses=[]

#100회 반복
for e in range(100):
    opti.zero_grad()
    
    #y 값 예측
    y_pred=net(X)
    
    # MSE loss, w grad 계산
    loss=loss_fn(y_pred.view_as(y),y)
    loss.backward()
    
    # 경사 갱신(계산 그래프 유지를 위해 .data만 갱신
    opti.step()
    
    #수렴확인을 위한 loss기록
    losses.append(loss.item())

In [6]:
list(net.parameters())

[Parameter containing:
 tensor([[0.9178, 1.9964, 3.0660]], requires_grad=True)]

# 로지스틱 회귀 분석

In [22]:
import torch
from torch import nn, optim
from sklearn.datasets import load_iris
iris=load_iris()

# 3가지의 꽃 종류중 두가지만 사용
x=iris.data[:100]
y=iris.target[:100]

#tensor 변환
x=torch.tensor(x,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.float32)

#iris 데이터는 4차원
net2=nn.Linear(4,1)

#두 클래스의 분류를 위한 크로스 엔트로피 계산
loss_fn2=nn.BCEWithLogitsLoss() 

optim=optim.SGD(net.parameters(),lr=0.25)


In [23]:
losses=[]

#100회 반복
for e in range(100):
    optim.zero_grad()
    
    #y 값 예측
    y_pred=net2(x)
    
    # MSE loss, w grad 계산
    loss=loss_fn2(y_pred.view_as(y),y)
    loss.backward()
    
    # 경사 갱신(계산 그래프 유지를 위해 .data만 갱신
    optim.step()
    
    #수렴확인을 위한 loss기록
    losses.append(loss.item())

In [24]:
list(net2.parameters())

[Parameter containing:
 tensor([[-0.0404, -0.2450, -0.4094, -0.1405]], requires_grad=True),
 Parameter containing:
 tensor([0.4146], requires_grad=True)]

In [25]:
h=net2(x)

prob=nn.functional.sigmoid(h)

y_pred=prob>0.5

(y.byte()==y_pred.view_as(y)).sum().item()

50

# 다중분류

In [28]:
import torch
from torch import nn,optim
from sklearn.datasets import load_digits
digits=load_digits()

x=digits.data
y=digits.target

x=torch.tensor(x,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.int64)

net=nn.Linear(x.size()[1],10)

loss_fn=nn.CrossEntropyLoss()

opti=optim.SGD(net.parameters(),lr=0.01)


In [31]:
losses=[]

#100회 반복
for e in range(100):
    opti.zero_grad()
    
    #y 값 예측
    y_pred=net(x)
    
    # MSE loss, w grad 계산
    loss=loss_fn(y_pred,y)
    loss.backward()
    
    # 경사 갱신(계산 그래프 유지를 위해 .data만 갱신
    opti.step()
    
    #수렴확인을 위한 loss기록
    losses.append(loss.item())

In [34]:
_,y_pred=torch.max(net(x),1)
(y_pred==y).sum().item()/len(y)

0.9493600445186422